## Setup

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from unityagents import UnityEnvironment
from agents.utils import get_config, smooth
from agents.agents import DDPGAgent, TD3Agent
from tqdm.auto import tqdm

In [2]:
# Set global variables
ACTOR_WEIGHTS_PATH = 'agents/weights/pretrained_actor.pt'
CRITIC_WEIGHTS_PATH = 'agents/weights/pretrained_critic.pt'

######################################## Define this as explained in README.md###########################################
REACHER_PATH = 'environment/reacher_windows/Reacher.exe'
#########################################################################################################################

NUM_EPISODES = 1000

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

In [3]:
# Initialize agent and environment
agent_config = get_config()
agent = TD3Agent(config=agent_config)
env = UnityEnvironment(file_name=REACHER_PATH)

# Get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


## Training Day

In [4]:
# A function used to play one episode with a given agent
def play_one(agent, env, brain_name, train_mode=True):
    
    # Initialize the return
    episode_return = 0
    
    # Initialize stepcounter
    step_counter = 0
    
    # Reset Noise
    agent.ou_noise.reset()
    
    # Start episode
    env_info = env.reset(train_mode=train_mode)[brain_name]
    observation = env_info.vector_observations[0]
    done = False
    
    # Play
    while not done:
        # Interact with environment
        action = agent.act(observation, step_counter)
        env_info = env.step(action)[brain_name]
        next_observation = env_info.vector_observations[0]
        reward = env_info.rewards[0]
        done = env_info.local_done[0]
        episode_return += reward     

        # Add memory
        agent.memento(observation, action, reward, next_observation, done)
        # Train from memory
        if agent.memory.is_ready():
            agent.learn()
        
        # Perform transition
        observation = next_observation
    
    return episode_return

In [ ]:
# Populating replay buffer:
env_info = env.reset(train_mode=True)[brain_name]
observation = env_info.vector_observations[0]
done = False

print('Filling Replay Buffer...')
while not agent.memory.is_ready():
    action = np.clip(np.random.randn(4), -1, 1)
    env_info = env.step(action)[brain_name]
    reward = env_info.rewards[0]
    next_observation = env_info.vector_observations[0]
    done = env_info.local_done[0]

    # Add memory
    agent.memento(observation, action, reward, next_observation, done)
    
    observation = next_observation
    if done:
        env_info = env.reset(train_mode=True)[brain_name]
        observation = env_info.vector_observations[0]

# Now we are able to train our agent
episode_returns = np.zeros(NUM_EPISODES)

# Activate agent's train mode
agent.train_mode = True

print('Starting Training...')
for i in tqdm(range(NUM_EPISODES)):
    episode_returns[i] = play_one(agent, env, brain_name)
    if i % (NUM_EPISODES // 10) == 0 and i > 0:
        print(f'Average Return over the last 100 episodes: {smooth(episode_returns[:i+1])[-1]}')

Filling Replay Buffer...
Starting Training...


Average Return over the last 100 episodes: 0.5581818057054823
Average Return over the last 100 episodes: 0.540952368861153
Average Return over the last 100 episodes: 0.8325806265516628
Average Return over the last 100 episodes: 1.1685365592470256
Average Return over the last 100 episodes: 1.4292156543290497
Average Return over the last 100 episodes: 1.619180291677352
Average Return over the last 100 episodes: 2.116760516066996
Average Return over the last 100 episodes: 2.5916048803447205
Average Return over the last 100 episodes: 3.099670260387299
Average Return over the last 100 episodes: 3.4994999217800795
Average Return over the last 100 episodes: 4.346699902843684
Average Return over the last 100 episodes: 5.221299883294851
Average Return over the last 100 episodes: 5.853199869170785
Average Return over the last 100 episodes: 6.811599847748876
Average Return over the last 100 episodes: 7.533099831622094
Average Return over the last 100 episodes: 8.375499812792986
Average Return ove

In [ ]:
# Plot the results:
plt.figure(figsize=(20,7.5))
plt.plot(episode_returns, color='gainsboro', label='return per episode')
plt.plot(smooth(episode_returns), color='blue', label='return averaged over last 100 episodes')
plt.plot(np.ones(NUM_EPISODES)*30, color='black', label='Target return (30)')
plt.legend()

# Print the return over the last 100 episodes:
print(f'Over the last 100 episodes, the agent received an average return of {smooth(episode_returns)[-1]}')

In [ ]:
# Save weights
agent.save(ACTOR_WEIGHTS_PATH, CRITIC_WEIGHTS_PATH)

## Evaluate

In [ ]:
# Dectivate agent's train mode (setting exploration to zero)
agent.train_mode = False
# Play 100 episodes and print the average reward (repeated for 10 runs):
for i in tqdm(range(1)):
    cache = np.zeros(100)
    for j in range(100):
        cache[j] = play_one(agent, env, brain_name)
    print(f'In run {i+1} the Agent received an average return of {np.mean(cache)} over 100 episodes.')

## Watch

In [ ]:
# Watch Time
play_one(agent, env, brain_name, train_mode=False)

## Clean up

In [ ]:
# Shutdown
env.close()